In [1]:
import os
import findspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [2]:
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-21"
os.environ["SPARK_HOME"] = "C:\spark-3.5.0-bin-hadoop3"

In [3]:
findspark.init()
spark = SparkSession.builder.appName("Property crime Analysis Table").getOrCreate()

In [4]:
findspark.find()

'C:\\spark-3.5.0-bin-hadoop3'

In [5]:
spark

In [6]:
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1701608459990'),
 ('spark.app.submitTime', '1701608459739'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'Property crime Analysis Table'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectM

In [42]:
# Load datasets
chicago_climate_data = spark.read.csv("chicago_climate_data.csv", header=True, inferSchema=True)
housing_data = spark.read.csv("housing_zhvi_final.csv", header=True, inferSchema=True)
crime_data = spark.read.csv("crime_final.csv", header=True, inferSchema=True)

## Analyzing housing data for chicago

In [43]:
housing_data.printSchema()

root
 |-- RegionID: integer (nullable = true)
 |-- RegionName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ZHVI: double (nullable = true)



In [44]:
housing_data.describe()

DataFrame[summary: string, RegionID: string, RegionName: string, City: string, ZHVI: string]

In [45]:
housing_data.show(n=1)

+--------+----------+-------+------------------+
|RegionID|RegionName|   City|              ZHVI|
+--------+----------+-------+------------------+
|  137634|  Burnside|Chicago|139033.83333333334|
+--------+----------+-------+------------------+
only showing top 1 row



In [46]:
housing_data.take(5)

[Row(RegionID=137634, RegionName='Burnside', City='Chicago', ZHVI=139033.83333333334),
 Row(RegionID=137841, RegionName='Clearing', City='Chicago', ZHVI=246129.83333333334),
 Row(RegionID=138129, RegionName='Dunning', City='Chicago', ZHVI=294552.4166666667),
 Row(RegionID=138166, RegionName='East Side', City='Chicago', ZHVI=132832.33333333334),
 Row(RegionID=138261, RegionName='Englewood', City='Chicago', ZHVI=53393.166666666664)]

In [77]:
housing_data.columns

['RegionID', 'RegionName', 'City', 'ZHVI']

In [47]:
housing_data.show()

+--------+------------------+-------+------------------+
|RegionID|        RegionName|   City|              ZHVI|
+--------+------------------+-------+------------------+
|  137634|          Burnside|Chicago|139033.83333333334|
|  137841|          Clearing|Chicago|246129.83333333334|
|  138129|           Dunning|Chicago| 294552.4166666667|
|  138166|         East Side|Chicago|132832.33333333334|
|  138261|         Englewood|Chicago|53393.166666666664|
|  138732|         Hegewisch|Chicago|          135495.5|
|  139408|     McKinley Park|Chicago| 263932.9166666667|
|  140024|           Pullman|Chicago|133982.91666666666|
|  140437|     South Chicago|Chicago|         117142.25|
|  140440|     South Deering|Chicago|104284.58333333333|
|  140913|      West Pullman|Chicago| 92588.83333333333|
|  269566|       Albany Park|Chicago| 344270.6666666667|
|  269568|    Archer Heights|Chicago|          217370.5|
|  269570|           Ashburn|Chicago|          197536.0|
|  269571|           Gresham|Ch

## Analyzing crima data for chicago

In [48]:
crime_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Year: integer (nullable = true)
 |-- regionid: double (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- population: integer (nullable = true)



In [49]:
crime_data.take(2)

[Row(ID=12098557, Case Number='JD288415', Date=datetime.datetime(2019, 2, 1, 0, 1), Primary Type='BATTERY', Description='SIMPLE', Location Description='RESIDENCE', Arrest=False, Domestic=True, Year=2019, regionid=269582.0, neighborhood='Gage Park', population=39894),
 Row(ID=12073110, Case Number='JD258449', Date=datetime.datetime(2019, 11, 1, 8, 0), Primary Type='BURGLARY', Description='UNLAWFUL ENTRY', Location Description='CHURCH / SYNAGOGUE / PLACE OF WORSHIP', Arrest=False, Domestic=False, Year=2019, regionid=269582.0, neighborhood='Gage Park', population=39894)]

In [50]:
from pyspark.sql.functions import col

In [51]:
from pyspark.sql.types import IntegerType

In [52]:
crime_data.select(col("Year").cast(IntegerType())).printSchema()

root
 |-- Year: integer (nullable = true)



In [53]:
crime_data.take(1)

[Row(ID=12098557, Case Number='JD288415', Date=datetime.datetime(2019, 2, 1, 0, 1), Primary Type='BATTERY', Description='SIMPLE', Location Description='RESIDENCE', Arrest=False, Domestic=True, Year=2019, regionid=269582.0, neighborhood='Gage Park', population=39894)]

In [54]:
crime_data.show()

+--------+-----------+-------------------+--------------------+--------------------+--------------------+------+--------+----+--------+------------+----------+
|      ID|Case Number|               Date|        Primary Type|         Description|Location Description|Arrest|Domestic|Year|regionid|neighborhood|population|
+--------+-----------+-------------------+--------------------+--------------------+--------------------+------+--------+----+--------+------------+----------+
|12098557|   JD288415|2019-02-01 00:01:00|             BATTERY|              SIMPLE|           RESIDENCE| false|    true|2019|269582.0|   Gage Park|     39894|
|12073110|   JD258449|2019-11-01 08:00:00|            BURGLARY|      UNLAWFUL ENTRY|CHURCH / SYNAGOGU...| false|   false|2019|269582.0|   Gage Park|     39894|
|12079127|   JD265413|2019-04-01 22:20:00|         SEX OFFENSE|AGGRAVATED CRIMIN...|           APARTMENT| false|   false|2019|269582.0|   Gage Park|     39894|
|11715936|   JC298110|2019-06-08 19:45:0

In [55]:
crime_data.select(["Case Number", "Year", "ID", "neighborhood"]).where(crime_data.ID == "24456").collect()

[Row(Case Number='JC204604', Year=2019, ID=24456, neighborhood='Gage Park')]

In [56]:
crime_data.count()

256686

## Analyzing Climate data for chicago

In [57]:
chicago_climate_data.printSchema()

root
 |-- index: integer (nullable = true)
 |-- dt: date (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [58]:
chicago_climate_data.take(2)

[Row(index=51674, dt=datetime.date(1743, 11, 1), day=1, month=11, year=1743, City='Chicago', AverageTemperature=5.436, AverageTemperatureUncertainty=2.205, Latitude='42.59N', Longitude='87.27W'),
 Row(index=51675, dt=datetime.date(1743, 12, 1), day=1, month=12, year=1743, City='Chicago', AverageTemperature=None, AverageTemperatureUncertainty=None, Latitude='42.59N', Longitude='87.27W')]

In [59]:
chicago_climate_data.groupBy("year").count().show()

+----+-----+
|year|count|
+----+-----+
|1829|   12|
|1959|   12|
|1896|   12|
|1990|   12|
|1903|   12|
|1884|   12|
|1975|   12|
|1977|   12|
|1766|   12|
|1787|   12|
|1888|   12|
|1808|   12|
|1743|    2|
|1863|   12|
|1924|   12|
|1745|   12|
|1823|   12|
|2003|   12|
|1869|   12|
|2007|   12|
+----+-----+
only showing top 20 rows



In [89]:
chicago_climate_data.columns

['index',
 'dt',
 'day',
 'month',
 'year',
 'City',
 'AverageTemperature',
 'AverageTemperatureUncertainty',
 'Latitude',
 'Longitude']

In [60]:
chicago_climate_data.sort(chicago_climate_data.year.desc()).show(truncate=False)

+-----+----------+---+-----+----+-------+------------------+-----------------------------+--------+---------+
|index|dt        |day|month|year|City   |AverageTemperature|AverageTemperatureUncertainty|Latitude|Longitude|
+-----+----------+---+-----+----+-------+------------------+-----------------------------+--------+---------+
|54907|2013-04-01|1  |4    |2013|Chicago|6.644             |0.462                        |42.59N  |87.27W   |
|54904|2013-01-01|1  |1    |2013|Chicago|1.056             |0.528                        |42.59N  |87.27W   |
|54905|2013-02-01|1  |2    |2013|Chicago|-0.509            |0.473                        |42.59N  |87.27W   |
|54906|2013-03-01|1  |3    |2013|Chicago|1.3230000000000002|0.275                        |42.59N  |87.27W   |
|54908|2013-05-01|1  |5    |2013|Chicago|13.734000000000002|0.863                        |42.59N  |87.27W   |
|54909|2013-06-01|1  |6    |2013|Chicago|17.913            |0.669                        |42.59N  |87.27W   |
|54910|201

In [61]:
chicago_climate_data.show()

+-----+----------+---+-----+----+-------+------------------+-----------------------------+--------+---------+
|index|        dt|day|month|year|   City|AverageTemperature|AverageTemperatureUncertainty|Latitude|Longitude|
+-----+----------+---+-----+----+-------+------------------+-----------------------------+--------+---------+
|51674|1743-11-01|  1|   11|1743|Chicago|             5.436|                        2.205|  42.59N|   87.27W|
|51675|1743-12-01|  1|   12|1743|Chicago|              NULL|                         NULL|  42.59N|   87.27W|
|51676|1744-01-01|  1|    1|1744|Chicago|              NULL|                         NULL|  42.59N|   87.27W|
|51677|1744-02-01|  1|    2|1744|Chicago|              NULL|                         NULL|  42.59N|   87.27W|
|51678|1744-03-01|  1|    3|1744|Chicago|              NULL|                         NULL|  42.59N|   87.27W|
|51679|1744-04-01|  1|    4|1744|Chicago|             8.766|                        2.357|  42.59N|   87.27W|
|51680|174

## Analyzing Combine data of crime and housing saved as  final_data.csv

In [83]:
crime_housing_combine = spark.read.csv("final_data.csv", header=True, inferSchema=True)

In [84]:
crime_housing_combine.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Year: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- RegionID: integer (nullable = true)
 |-- RegionName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ZHVI: double (nullable = true)



In [85]:
crime_housing_combine.take(2)

[Row(ID=12098557, Case Number='JD288415', Date=datetime.datetime(2019, 2, 1, 0, 1), Primary Type='BATTERY', Description='SIMPLE', Location Description='RESIDENCE', Arrest=False, Domestic=True, Year=2019, population=39894, RegionID=269582, RegionName='Gage Park', City='Chicago', ZHVI=185517.41666666666),
 Row(ID=12073110, Case Number='JD258449', Date=datetime.datetime(2019, 11, 1, 8, 0), Primary Type='BURGLARY', Description='UNLAWFUL ENTRY', Location Description='CHURCH / SYNAGOGUE / PLACE OF WORSHIP', Arrest=False, Domestic=False, Year=2019, population=39894, RegionID=269582, RegionName='Gage Park', City='Chicago', ZHVI=185517.41666666666)]

In [86]:
crime_housing_combine.show()

+--------+-----------+-------------------+--------------------+--------------------+--------------------+------+--------+----+----------+--------+----------+-------+------------------+
|      ID|Case Number|               Date|        Primary Type|         Description|Location Description|Arrest|Domestic|Year|population|RegionID|RegionName|   City|              ZHVI|
+--------+-----------+-------------------+--------------------+--------------------+--------------------+------+--------+----+----------+--------+----------+-------+------------------+
|12098557|   JD288415|2019-02-01 00:01:00|             BATTERY|              SIMPLE|           RESIDENCE| false|    true|2019|     39894|  269582| Gage Park|Chicago|185517.41666666666|
|12073110|   JD258449|2019-11-01 08:00:00|            BURGLARY|      UNLAWFUL ENTRY|CHURCH / SYNAGOGU...| false|   false|2019|     39894|  269582| Gage Park|Chicago|185517.41666666666|
|12079127|   JD265413|2019-04-01 22:20:00|         SEX OFFENSE|AGGRAVATED C

In [87]:
crime_housing_combine.select(col("Primary Type").cast(IntegerType())).printSchema()

root
 |-- Primary Type: integer (nullable = true)



In [88]:
crime_housing_combine.columns

['ID',
 'Case Number',
 'Date',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Year',
 'population',
 'RegionID',
 'RegionName',
 'City',
 'ZHVI']

### Combining all data

In [78]:
spark = SparkSession.builder.appName("JoinDatasets").getOrCreate()

In [91]:
# Join datasets based on the 'City' column
joined_data = crime_housing_combine.join(chicago_climate_data, 'City', 'inner')

In [92]:
joined_data.show()

+-------+--------+-----------+-------------------+------------+-----------+--------------------+------+--------+----+----------+--------+----------+------------------+-----+----------+---+-----+----+------------------+-----------------------------+--------+---------+
|   City|      ID|Case Number|               Date|Primary Type|Description|Location Description|Arrest|Domestic|Year|population|RegionID|RegionName|              ZHVI|index|        dt|day|month|year|AverageTemperature|AverageTemperatureUncertainty|Latitude|Longitude|
+-------+--------+-----------+-------------------+------------+-----------+--------------------+------+--------+----+----------+--------+----------+------------------+-----+----------+---+-----+----+------------------+-----------------------------+--------+---------+
|Chicago|12098557|   JD288415|2019-02-01 00:01:00|     BATTERY|     SIMPLE|           RESIDENCE| false|    true|2019|     39894|  269582| Gage Park|185517.41666666666|54912|2013-09-01|  1|    9|20